## Scatter-Gather Pattern

This notebook demonstrates the **Scatter-Gather** Enterprise Integration Pattern (EIP) using Rustic AI agents. 

The Scatter-Gather pattern sends a message to multiple recipients and aggregates the responses back into a single message. This is useful for:
- Collecting data from multiple sources
- Running parallel analysis tasks  
- Distributed processing with consolidation

### Pattern Flow

**1→3→1 Message Flow:**

```
[ProbeAgent] --AnalysisRequest--> (analysis_requests) 
                    ↓ SCATTER PHASE
    [ScatterAgent] routes to 3 specialized agents:
    ├── StatisticsAgent: Computes mean, median, std dev
    ├── TrendAgent: Analyzes data trends and slopes  
    └── AnomalyAgent: Detects outliers using 2-sigma rule
                    ↓ GATHER PHASE
    [AggregatingAgent] collects all 3 responses by correlation_id
                    ↓ 
    [ReportAgent] generates comprehensive analysis report
```

### Key Components

- **ScatterAgent**: Routes analysis requests to multiple specialized agents
- **AggregatingAgent**: Collects and correlates responses using the new EIP component
- **Collector Strategies**: Uses `DictCollector` to deduplicate by analysis type
- **Correlation Tracking**: Maintains request-response correlation across the flow

### Message Types

First, let's define the message models for our scatter-gather pattern. We have:
- **Request messages**: For scattering to different analysis agents
- **Result messages**: For gathering responses back
- **Report message**: For the final comprehensive analysis

Look at [scatter_gather_models.py](./helpers/scatter_gather_models.py) for the messages.

### Analysis and Report Generation Agents

These are the specialized domain specific agents that perform duties for the overall analysis task.

There are 3 Analysis Agents:
- Statistical Analysis
- Trend Analysis
- Anomaly Analysis

And one Reporting Agent.

Look at [scatter_gather_agents.py](./helpers/scatter_gather_agents.py) for details.

### Guild Creation

Now let's create the guild that orchestrates our scatter-gather pattern with all agents and routing rules.

In [ ]:
# Build the scatter-gather guild with bootstrap for state management
import os

from rustic_ai.core.guild.builders import GuildBuilder
from rustic_ai.core.guild.metastore.database import Metastore

guild_builder = GuildBuilder.from_yaml_file("./006_scatter_gather.yaml")

# Bootstrap the guild so we have GuildManager for state management
db = "sqlite:///scatter_gather_demo.db"
# if file exists db delete it


if os.path.exists("scatter_gather_demo.db"):
    os.remove("scatter_gather_demo.db")

Metastore.initialize_engine(db)
Metastore.get_engine(db)
Metastore.create_db()
guild = guild_builder.bootstrap(metastore_database_url=db, organization_id="myorg")  # Use SQLite for simplicity

### Probe Agent Setup

Let's create a probe agent to monitor the entire scatter-gather flow and test the pattern.

In [ ]:
import time

from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent
from rustic_ai.core.guild.builders import AgentBuilder

time.sleep(2)

# Create probe agent to monitor the entire flow
probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("TestProbe")
    .set_name("Test Probe Agent")
    .set_description("Monitors the entire scatter-gather flow")
    .add_additional_topic("analysis_requests")  # Initial requests
    .add_additional_topic("statistics_analysis")  # Scattered requests
    .add_additional_topic("trend_analysis")
    .add_additional_topic("anomaly_analysis")
    .add_additional_topic("analysis_results")  # Individual analysis results
    .add_additional_topic("aggregated_results")  # Aggregated results
    .add_additional_topic("final_reports")  # Final comprehensive reports
    .build()
)

# Add probe agent to the bootstrapped guild
guild._add_local_agent(probe_agent)

In [ ]:
from rustic_ai.core.agents.testutils.probe_agent import EssentialProbeAgent
from rustic_ai.core.guild.dsl import GuildTopics


system_probe = (
    AgentBuilder(EssentialProbeAgent)
    .set_id("SystemProbe")
    .set_name("System Probe Agent")
    .set_description("Monitors the system topic")
    .add_additional_topic(GuildTopics.SYSTEM_TOPIC)  # Subscribe to system messages
    .build()
)

### Testing the Scatter-Gather Pattern

Now let's test our complete scatter-gather implementation with sample data.

In [ ]:
# Create test data for analysis
import uuid

from helpers.scatter_gather_models import AnalysisRequest

test_request = AnalysisRequest(
    request_id=str(uuid.uuid4()),
    dataset=[100.0, 120.0, 95.0, 110.0, 130.0, 85.0, 140.0, 105.0],
    query="Comprehensive data analysis",
)

print(f"Sending AnalysisRequest with ID: {test_request.request_id}")
print(f"Dataset: {test_request.dataset}")

# Send through probe agent to trigger the scatter-gather flow
probe_agent.publish_with_guild_route(topic="analysis_requests", payload=test_request)

In [ ]:
# Let's check the message history to see the complete flow
probe_agent.print_all_history()

In [ ]:
probe_agent.get_messages()

In [ ]:
probe_agent._client._messaging.get_messages_for_topic_since(
    topic=GuildTopics.STATE_TOPIC,
    msg_id_since=0
)

### Pattern Summary

This notebook demonstrated the **Scatter-Gather** Enterprise Integration Pattern using Rustic AI:

#### Key Features Implemented:
1. **Scatter Phase**: Single `AnalysisRequest` routed to 3 specialized analysis agents
2. **Parallel Processing**: Statistics, trend, and anomaly analysis run concurrently  
3. **Gather Phase**: `AggregatingAgent` correlates and collects all responses
4. **Final Processing**: `ReportAgent` creates comprehensive analysis report

#### EIP Components Used:
- **BasicWiringAgent**: For message routing and scattering
- **AggregatingAgent**: For correlation-based message aggregation
- **DictCollector**: For deduplication by analysis type
- **CountingAggregator**: To wait for exactly 3 responses
- **Payload Transformers**: JxScript transformations for message adaptation

#### Message Flow:
```
1 Request → 3 Parallel Analyses → 1 Aggregated Result → 1 Final Report
```

This pattern is essential for distributed processing scenarios where you need to:
- Collect data from multiple sources
- Run parallel computations
- Aggregate results maintaining correlation
- Generate consolidated reports

The implementation showcases reusable EIP components that can be adapted for various scatter-gather scenarios.

In [ ]:
from rustic_ai.core.agents.system.models import StopGuildRequest


probe_agent.publish_with_guild_route(
    topic=GuildTopics.SYSTEM_TOPIC, payload=StopGuildRequest(guild_id=guild.id)
)  # Send the test request to start the flow

guild.shutdown()